# Training, Engineering, and Model Assesment

## Import packages 

In [37]:
# general
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

# sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor


# plotting
import matplotlib.pyplot as plt
import seaborn as sns

## Import Data

In [38]:
# set lists of coordinates and time ranges (pulled from Prepare_AI_Ready_Data.py) (CURRENTLY JUST ONE, BUT CAN ADD MORE)
coords = [[180,240,45,65],[130,250,20,75]]
times = [['1970-01-01','2023-12-31']]  # Ensure the time range is valid

# select which of the list I want to load
coords_num = 1
times_num = 0

# pull the correct coordinate and time (as set above)
c = coords[coords_num]
t = times[times_num]

# Import Target Data
pc_df_raw = pd.read_csv(f'../data/dimensionality_reduction/principal_components_{c[0]}-{c[1]}_{c[2]}-{c[3]}_{t[0][:4]}-{t[1][:4]}_target.csv')


## Linear Regression

I already have a pretty good sense that a linear regression isn't going to work well. We know the month is important, but a linear regression won't handle 1 (Jan) being much 'closer' to 12 (Dec) than 4 (April) very well

In [39]:
data = pc_df_raw

# Splitting the data into independent and dependent variables
X = data[['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8','PC9','PC10', 'month']]
Y = data['pwt_500hpa']

# Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.3, random_state=42
)

# Fitting the model
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Calculating the test set results
y_pred = regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"mean square error {mse} and R2 {r2}")


mean square error 23.318611004978756 and R2 0.0904208591182264


As expected, the preformance of the linear regression above is pretty weak. Now let's try modifying the 'month' column to give this a chance - we'll make it months from the hotest month time of year (July/August). July/August will be a '0', June/Sepetember a 1, and so on

In [40]:
# Adding a new column to the data where we now caluclate the difference between the month and 7 and 8
data['month_modified'] = data['month'].apply(lambda x: min(abs(x - 7), abs(x - 8)))

In [41]:
# Splitting the data into independent and dependent variables
X = data[['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8','PC9','PC10', 'month_modified']]
Y = data['pwt_500hpa']

# Splitting the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.3, random_state=42
)

# Fitting the model
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Calculating the test set results
y_pred = regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"mean square error {mse} and R2 {r2}")

mean square error 8.880168678503813 and R2 0.653615037544296


Wow, this was a big improvement! We jumped from r^2 <0.1 to r^2 ~ 0.65. This points to the importance of having a physically interpretable month. However, this r^2 value is still significantly worse than what I was achieving in the auto_ml notebook. Rather than further develop/test the linear model, I'm going to switch to some more complex approaches to see what I can do.

## Random Forest Regression

Here I train a random forest regression. This was one of the most promising approaches identified in the AutoMl analysis.

The first thing I'm going to do is use one-hot encoding on the month column. This reduces the risk of assinging too much importance to a Dec (12) rather than a Jan (1)

In [42]:
# Preform one-hot encoding on the month column
data_onehot = pd.get_dummies(data, columns=['month'])

# confirm result
data_onehot.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,-33110.538572,82249.847995,-21333.242489,-1453.484937,21087.121592,12198.921330,-14981.360841,-4590.483890,-2802.138055,-10041.679572,...,False,False,False,False,False,False,False,False,False,False
1,54023.011927,39062.768941,16181.108367,-29075.800303,1099.333227,13881.630520,-8568.927938,1504.575300,5601.959437,-1887.312548,...,False,False,False,False,False,False,False,False,False,False
2,37578.387648,43678.134428,9401.044754,-42940.839537,-10443.023957,-8730.349263,6792.424158,-1706.251578,9137.777188,2764.929788,...,True,False,False,False,False,False,False,False,False,False
3,-8043.467741,-27123.081043,-8808.002732,-22731.351138,-520.103014,-5128.049218,17417.902610,-10787.543733,3462.505202,-5885.287505,...,False,True,False,False,False,False,False,False,False,False
4,-15116.705639,-724.941120,-9204.978768,5316.306251,-855.566249,11667.794125,16685.172843,1335.961037,1177.432451,3893.232298,...,False,False,True,False,False,False,False,False,False,False


In [43]:
# Use numpy to convert to arrays
import numpy as np

# Labels are the values we want to predict
labels = np.array(data_onehot['pwt_500hpa'])

# Remove the labels from the features
# axis 1 refers to the columns
features = data_onehot[['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8','PC9','PC10', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12']]

# Saving feature names for later use
feature_list = list(features.columns)

# Convert to numpy array
features = np.array(features)

# split the data into test/training sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.3, random_state = 42)

# establish the model baseline. We'll make averages over the time the baseline prediction
baseline_preds = np.mean(test_labels)



In [44]:
# start timer
t1 = time.perf_counter()

# initalize the model
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf.fit(train_features, train_labels)

# save training time
t2 = time.perf_counter()
print(f"Training time: {t2-t1} seconds")
t_rf = t2-t1


KeyboardInterrupt: 

In [ ]:
# assess accuracy

# Use the forest's predict method on the test data
predictions = rf.predict(test_features)

# Calculate the absolute errors
errors = abs(predictions - test_labels)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]


Mean Absolute Error: 1.67
Accuracy: 99.32 %.
Variable: month_7              Importance: 0.24
Variable: month_8              Importance: 0.21
Variable: month_6              Importance: 0.16
Variable: month_9              Importance: 0.13
Variable: month_10             Importance: 0.05
Variable: month_5              Importance: 0.04
Variable: PC3                  Importance: 0.03
Variable: PC2                  Importance: 0.02
Variable: PC5                  Importance: 0.02
Variable: PC7                  Importance: 0.02
Variable: PC9                  Importance: 0.02
Variable: PC1                  Importance: 0.01
Variable: PC4                  Importance: 0.01
Variable: PC6                  Importance: 0.01
Variable: PC8                  Importance: 0.01
Variable: PC10                 Importance: 0.01
Variable: month_3              Importance: 0.01
Variable: month_1              Importance: 0.0
Variable: month_2              Importance: 0.0
Variable: month_4              Importance: 0.

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

Now that we've assessed a single model, let's preform a leave out cross validation. We have a relatively small dataset, and can afford this computationally intesive approach. In return, we'll get a very accuarate picture of our model's predicitive capacity.

In [ ]:
from tqdm import tqdm
from sklearn.model_selection import LeaveOneOut

# Initialize Leave-One-Out cross-validator
loo = LeaveOneOut()

# Initialize variables to store results
errors = []
accuracies = []
feature_importances_list = []

# Start timer
t1 = time.perf_counter()

# Perform Leave-One-Out cross-validation with a progress bar
for train_index, test_index in tqdm(loo.split(features), total=features.shape[0], desc="LOO Progress"):
    # Split data
    train_features, test_features = features[train_index], features[test_index]
    train_labels, test_labels = labels[train_index], labels[test_index]
    
    # Initialize the model
    rf = RandomForestRegressor(n_estimators=1000, random_state=42)
    
    # Train the model
    rf.fit(train_features, train_labels)
    
    # Predict on the test set
    predictions = rf.predict(test_features)
    
    # Calculate the absolute error
    error = abs(predictions - test_labels)
    errors.append(error[0])
    
    # Calculate accuracy
    mape = 100 * (error / test_labels)
    accuracy = 100 - mape
    accuracies.append(accuracy[0])
    
    # Store feature importances
    feature_importances_list.append(rf.feature_importances_)

# End timer
t2 = time.perf_counter()

# Calculate average error and accuracy
mean_error = np.mean(errors)
mean_accuracy = np.mean(accuracies)

# Calculate average feature importances
mean_feature_importances = np.mean(feature_importances_list, axis=0)

# Print results
print(f"Mean Absolute Error (LOO): {round(mean_error, 2)}")
print(f"Mean Accuracy (LOO): {round(mean_accuracy, 2)}%")
print(f"Cross-validation time: {t2 - t1} seconds")


LOO Progress:   0%|          | 0/636 [00:00<?, ?it/s]